# Pre-processing

In [ ]:
####################################
##        ACTION RECQUIRED        ##
####################################

# Update the following line with the starting and ending nodes of your choice:
start_node, end_node = 50, 10

####################################
##    END OF ACTION RECQUIRED     ##
####################################


# Defining custom functions
def formatInput(raw_file):
    ## Creating key-value pairs: key = node A ; value = (node B, distance from A to B)
    file = raw_file.map(lambda x: (x.split('\t')[0], (x.split('\t')[2], x.split('\t')[1])))
    
    ## Grouping by key
    file = file.groupByKey()
    
    ## Adding the distance (1) and the path (2)
    file = file.map(lambda x: (x[0], 
                               0 if x[0] == str(start_node) else float('inf'), ##(1)
                               list(x[1]),
                               " ")) ## (2)
    return file


# Transforming our file
file = sc.textFile("../data/graph.txt")
file = formatInput(file)
sc._jsc.hadoopConfiguration().set('mapreduce.fileoutputcommitter.marksuccessfuljobs', 'false') # Preventing "SUCCESS" file to be created when writting a text file
try:
    file.coalesce(1).saveAsTextFile("input")
except:
    print('Error: "Input" directory already exists. You need to remove it.')

**Example of output after transformation:**

<u>for "file.take(2)"</u>: [('4', inf, [('274', '5247'), ('766', '3762'), ('74', '8870'), ('23', '2656'), ('762', '5391'), ('142', '3150'), ('782', '5363'), ('394', '5098')], ' '), ('64', inf, [('5', '1007'), ('314', '2250'), ('314', '1541'), ('446', 
'7225'), ('11', '180'), ('751', '3919'), ('17', '9480'), ('508', '6730'), ('320', '9639'), ('58', '8591')], ' ')]